In [ ]:
import json
import os
import requests
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

# Define the Rasa NLU endpoint URL
nlu_url = "http://localhost:5005/model/parse"

# Define the dashboard app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div(children=[
    html.H1(children='NLQ Dashboard'),
    html.Div(children='Enter your query:'),
    dcc.Input(id='query-input', value='', type='text'),
    html.Button('Submit', id='submit-button', n_clicks=0),
    html.Div(id='output')
])

# Define the NLQ function
def nlq(query):
    # Define the payload for the Rasa NLU API
    payload = {
        "text": query
    }

    # Send the query to the Rasa NLU API
    response = requests.post(nlu_url, json=payload)

    # Parse the response from the Rasa NLU API
    response_json = json.loads(response.text)
    intent = response_json['intent']['name']
    entities = response_json['entities']

    # Process the intent and entities
    if intent == "get_sales":
        # TODO: replace this with code to query the sales data using the entities
        data = pd.read_csv('sales_data.csv')
        return data.to_json(orient='split')
    elif intent == "get_inventory":
        # TODO: replace this with code to query the inventory data using the entities
        data = pd.read_csv('inventory_data.csv')
        return data.to_json(orient='split')
    else:
        return "Sorry, I don't understand your query."

# Define the callback function for the submit button
@app.callback(
    Output('output', 'children'),
    Input('submit-button', 'n_clicks'),
    State('query-input', 'value')
)
def update_output(n_clicks, value):
    if n_clicks > 0:
        data = nlq(value)
        if type(data) == str:
            return data
        else:
            return dcc.Graph(
                id='graph',
                figure={
                    'data': [
                        {
                            'x': data['columns'],
                            'y': data['data'][0],
                            'type': 'bar',
                            'name': 'Data'
                        }
                    ],
                    'layout': {
                        'title': 'Dashboard'
                    }
                }
            )

# Run the dashboard app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
import json
import os
import requests
from opennlp.tools.tokenize import SimpleTokenizer

# Define the Rasa NLU endpoint URL
nlu_url = "http://localhost:5005/model/parse"

# Define the OpenNLP tokenizer
tokenizer = SimpleTokenizer()

# Define the dashboard data
sales_data = {
    "Q1": 100000,
    "Q2": 150000,
    "Q3": 200000,
    "Q4": 250000
}

inventory_data = {
    "product1": 500,
    "product2": 1000,
    "product3": 1500,
    "product4": 2000
}

# Define the NLQ function
def nlq(query):
    # Define the payload for the Rasa NLU API
    payload = {
        "text": query
    }

    # Send the query to the Rasa NLU API
    response = requests.post(nlu_url, json=payload)

    # Parse the response from the Rasa NLU API
    response_json = json.loads(response.text)
    intent = response_json['intent']['name']
    entities = response_json['entities']

    # Process the intent and entities
    if intent == "get_sales":
        # Get the time period from the query
        time_period = ""
        for token in tokenizer.tokenize(query):
            if token in sales_data:
                time_period = token
                break

        # Generate the dashboard based on the time period
        if time_period in sales_data:
            sales_amount = sales_data[time_period]
            dashboard = f"Sales for {time_period}: ${sales_amount}"
        else:
            dashboard = "Sorry, I couldn't find sales data for that time period."
    elif intent == "get_inventory":
        # Get the product from the query
        product = ""
        for entity in entities:
            if entity['entity'] == "product":
                product = entity['value']
                break

        # Generate the dashboard based on the product
        if product in inventory_data:
            inventory_amount = inventory_data[product]
            dashboard = f"Inventory for {product}: {inventory_amount} units"
        else:
            dashboard = "Sorry, I couldn't find inventory data for that product."
    else:
        dashboard = "Sorry, I don't understand your query."

    return dashboard

# Example usage
query = "What were our sales in Q1?"
dashboard = nlq(query)
print(dashboard)


TEXTBLOB

In [ ]:
# Import the necessary libraries
from textblob import TextBlob
import dash
import dash_core_components as dcc
import dash_html_components as html

# Define the list of sample queries and their corresponding dashboard options
queries = {'What is the total revenue for 2021?': 'Revenue dashboard',
           'Show me the sales performance by region': 'Sales performance dashboard',
           'What is the average order value for last month?': 'Order value dashboard'}

# Define the function to perform NLU on the user query
def process_query(query):
    # Perform NLU on the user query using TextBlob library
    blob = TextBlob(query)
    # Extract the noun phrases from the user query
    noun_phrases = [np.string for np in blob.noun_phrases]
    # Determine the dashboard option based on the noun phrases
    dashboard_option = 'default dashboard'
    for np in noun_phrases:
        if 'revenue' in np:
            dashboard_option = 'Revenue dashboard'
        elif 'sales' in np:
            dashboard_option = 'Sales performance dashboard'
        elif 'order value' in np:
            dashboard_option = 'Order value dashboard'
    return dashboard_option

# Create the Dash app
app = dash.Dash()

# Define the app layout
app.layout = html.Div([
    html.H1('NLQ Dashboard'),
    dcc.Input(id='query', value='', type='text'),
    html.Div(id='output')
])

# Define the callback function to process the user query and generate the dashboard
@app.callback(
    dash.dependencies.Output('output', 'children'),
    [dash.dependencies.Input('query', 'value')])
def update_output(query):
    dashboard_option = process_query(query)
    return dcc.Graph(
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'Option: {}'.format(dashboard_option)}
            ],
            'layout': {
                'title': 'Dashboard'
            }
        }
    )

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


RASA NLU

In [ ]:
# Import the necessary libraries
import json
from rasa.nlu.model import Interpreter
import dash
import dash_core_components as dcc
import dash_html_components as html

# Load the Rasa NLU model
interpreter = Interpreter.load('path/to/nlu/model')

# Define the list of sample queries and their corresponding dashboard options
queries = {'What is the total revenue for 2021?': 'Revenue dashboard',
           'Show me the sales performance by region': 'Sales performance dashboard',
           'What is the average order value for last month?': 'Order value dashboard'}

# Define the function to perform NLU on the user query
def process_query(query):
    # Perform NLU on the user query using Rasa NLU model
    result = interpreter.parse(query)
    # Extract the intent from the NLU result
    intent = result['intent']['name']
    # Determine the dashboard option based on the intent
    dashboard_option = 'default dashboard'
    if intent == 'revenue':
        dashboard_option = 'Revenue dashboard'
    elif intent == 'sales':
        dashboard_option = 'Sales performance dashboard'
    elif intent == 'order_value':
        dashboard_option = 'Order value dashboard'
    return dashboard_option

# Create the Dash app
app = dash.Dash()

# Define the app layout
app.layout = html.Div([
    html.H1('NLQ Dashboard'),
    dcc.Input(id='query', value='', type='text'),
    html.Div(id='output')
])

# Define the callback function to process the user query and generate the dashboard
@app.callback(
    dash.dependencies.Output('output', 'children'),
    [dash.dependencies.Input('query', 'value')])
def update_output(query):
    dashboard_option = process_query(query)
    return dcc.Graph(
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'Option: {}'.format(dashboard_option)}
            ],
            'layout': {
                'title': 'Dashboard'
            }
        }
    )

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


Apache OpenNLP

In [ ]:
# Import the necessary libraries
import json
from py4j.java_gateway import JavaGateway
from py4j.java_collections import ListConverter
import dash
import dash_core_components as dcc
import dash_html_components as html

# Start the Java Gateway
gateway = JavaGateway()

# Load the Apache OpenNLP model
model = gateway.jvm.opennlp.tools.models.PerceptronModel("path/to/nlp/model.bin")

# Define the list of sample queries and their corresponding dashboard options
queries = {'What is the total revenue for 2021?': 'Revenue dashboard',
           'Show me the sales performance by region': 'Sales performance dashboard',
           'What is the average order value for last month?': 'Order value dashboard'}

# Define the function to perform NLU on the user query
def process_query(query):
    # Create the OpenNLP components
    tokenizer = gateway.jvm.opennlp.tools.tokenize.SimpleTokenizer()
    pos_tagger = gateway.jvm.opennlp.tools.postag.POSTaggerME(model)
    parser = gateway.jvm.opennlp.tools.parser.ParserME(model.getParser())

    # Tokenize the user query
    tokens = tokenizer.tokenize(query)
    # Perform part-of-speech tagging on the tokens
    tags = pos_tagger.tag(tokens)
    # Parse the tagged tokens to get the constituent tree
    tree = parser.parse(ListConverter().convert(tokens))

    # Extract the noun phrases from the constituent tree
    noun_phrases = []
    for subtree in tree.subtree():
        if subtree.label().value() == "NP":
            noun_phrase = " ".join(word.value() for word in subtree.yield_)
            noun_phrases.append(noun_phrase)

    # Determine the dashboard option based on the noun phrases
    dashboard_option = 'default dashboard'
    for np in noun_phrases:
        if 'revenue' in np.lower():
            dashboard_option = 'Revenue dashboard'
        elif 'sales' in np.lower():
            dashboard_option = 'Sales performance dashboard'
        elif 'order value' in np.lower():
            dashboard_option = 'Order value dashboard'
    return dashboard_option

# Create the Dash app
app = dash.Dash()

# Define the app layout
app.layout = html.Div([
    html.H1('NLQ Dashboard'),
    dcc.Input(id='query', value='', type='text'),
    html.Div(id='output')
])

# Define the callback function to process the user query and generate the dashboard
@app.callback(
    dash.dependencies.Output('output', 'children'),
    [dash.dependencies.Input('query', 'value')])
def update_output(query):
    dashboard_option = process_query(query)
    return dcc.Graph(
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'Option: {}'.format(dashboard_option)}
            ],
            'layout': {
                'title': 'Dashboard'
            }
        }
    )

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
